In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)', 'Amapá - Produção de Cimento (t)'], axis=1)
data

,Ano,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Consumo de Cimento (t)
0,2003,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1046.256482,25.648717,22.351312,83.499469,0.994976,8.035813e+06,356591.003430,10.883143,7.033593e+06,85.0930
1,2004,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1094.465351,25.522298,22.367686,84.264642,1.022855,8.163242e+06,357271.494323,10.907898,7.078752e+06,80.0080
2,2005,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1171.421119,25.324126,22.395080,85.476824,1.065999,8.284323e+06,359035.074957,10.930315,7.120175e+06,104.1970
3,2006,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1248.347454,25.160986,22.452011,86.649032,1.102899,8.397333e+06,361349.457774,10.963941,7.156515e+06,115.5400
4,2007,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1296.469430,25.139657,22.556996,87.294293,1.112045,8.489305e+06,362177.947424,10.970195,7.171045e+06,135.0460
5,2008,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1287.012123,25.366918,22.728549,86.925632,1.071929,8.565394e+06,362905.280442,10.964829,7.173307e+06,116.3660
6,2009,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1230.481450,25.905555,23.095009,85.784466,0.981601,8.619980e+06,360014.231547,10.929234,7.163325e+06,112.7290
7,2010,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1208.062836,25.916677,23.199352,86.045978,0.939026,8.840356e+06,365333.788458,10.918810,7.227241e+06,151.5420
8,2011,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1245.286694,25.844771,23.091132,85.930050,0.977264,9.053310e+06,369799.968255,10.934033,7.289555e+06,161.9700
9,2012,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1288.178519,25.905589,23.183123,86.045636,1.031823,9.243435e+06,371290.647450,10.930025,7.338668e+06,261.1200


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,0.711421,8.514392,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,23.050698,1009.391930,1046.256482,25.648717,22.351312,83.499469,0.994976,8.035813e+06,356591.003430,10.883143,7.033593e+06
1,0.713006,8.446721,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,24.050121,1009.316288,1094.465351,25.522298,22.367686,84.264642,1.022855,8.163242e+06,357271.494323,10.907898,7.078752e+06
2,0.714376,8.415811,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,25.713548,1009.206027,1171.421119,25.324126,22.395080,85.476824,1.065999,8.284323e+06,359035.074957,10.930315,7.120175e+06
3,0.715533,8.334802,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,27.293226,1009.152758,1248.347454,25.160986,22.452011,86.649032,1.102899,8.397333e+06,361349.457774,10.963941,7.156515e+06
4,0.716480,8.318744,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,28.041427,1009.248097,1296.469430,25.139657,22.556996,87.294293,1.112045,8.489305e+06,362177.947424,10.970195,7.171045e+06
5,0.717218,8.281094,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,27.210421,1009.583656,1287.012123,25.366918,22.728549,86.925632,1.071929,8.565394e+06,362905.280442,10.964829,7.173307e+06
6,0.717750,8.302950,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,23.388804,1009.883647,1230.481450,25.905555,23.095009,85.784466,0.981601,8.619980e+06,360014.231547,10.929234,7.163325e+06
7,0.718079,8.270873,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,20.607778,1009.759833,1208.062836,25.916677,23.199352,86.045978,0.939026,8.840356e+06,365333.788458,10.918810,7.227241e+06
8,0.718205,8.258737,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,19.725663,1009.881525,1245.286694,25.844771,23.091132,85.930050,0.977264,9.053310e+06,369799.968255,10.934033,7.289555e+06
9,0.717668,14.118134,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,19.226938,1010.130132,1288.178519,25.905589,23.183123,86.045636,1.031823,9.243435e+06,371290.647450,10.930025,7.338668e+06


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
17    157.6210
18    142.7390
19         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-2.498491,-0.888884,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.341562,-1.041261,-2.622280,0.043210,-1.634720,-1.880012,-0.229055,-1.651592,-0.411311,-0.183091,-1.880195
1,-1.725700,-0.907434,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.070640,-1.181183,-1.944722,-0.382029,-1.591159,-1.405799,0.250560,-1.430423,-0.315776,0.264667,-1.501512
2,-1.057698,-0.915907,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,0.380277,-1.385145,-0.863135,-1.048620,-1.518280,-0.654555,0.992789,-1.220272,-0.068185,0.670127,-1.154156
3,-0.493413,-0.938113,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,0.808493,-1.483680,0.218038,-1.597376,-1.366821,0.071915,1.627596,-1.024129,0.256733,1.278344,-0.849422
4,-0.031772,-0.942515,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,1.011313,-1.307323,0.894375,-1.669122,-1.087520,0.471813,1.784948,-0.864499,0.373045,1.391456,-0.727572
5,0.328298,-0.952835,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,0.786046,-0.686608,0.761456,-0.904683,-0.631119,0.243337,1.094810,-0.732437,0.475156,1.294411,-0.708607
6,0.587870,-0.946844,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.249909,-0.131686,-0.033063,0.907138,0.343809,-0.463895,-0.459146,-0.637697,0.069279,0.650588,-0.792315
7,0.748015,-0.955637,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-1.003783,-0.360718,-0.348149,0.944550,0.621403,-0.301824,-1.191593,-0.255205,0.816097,0.462035,-0.256339
8,0.809807,-0.958964,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-1.242904,-0.135612,0.175020,0.702678,0.333494,-0.373670,-0.533758,0.114404,1.443108,0.737389,0.266205
9,0.547992,0.647212,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-1.378097,0.324259,0.777849,0.907252,0.578225,-0.302036,0.404846,0.444390,1.652386,0.664881,0.678053


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      80.0080
1     104.1970
2     115.5400
3     135.0460
4     116.3660
5     112.7290
6     151.5420
7     161.9700
8     261.1200
9     250.5190
10    227.2430
11    198.0696
12    166.8504
13    135.0210
14    123.3470
15    120.1530
16    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.388764,0.425197,-1.178467,-0.682869,0.106644,0.829078,-1.095947,0.821846,-1.025345,-1.444524,0.649078


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [15]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[610868037, 1720699993, 681176551, 4061057812, 953611386, 2662573014, 3193315962, 3957364853, 1864408410, 2545427542, 2989003777, 358013840, 2788538926, 2681685504, 3501321671, 978239468, 2116491476, 2128073553, 1775678847, 1368348078, 3288327858, 1536396656, 2445422041, 805116246, 939206821, 3146016668, 453547721, 346341569, 3013519845, 2490959567, 2182739872, 1877342387, 3490259277, 1600028871, 44878023, 3166660449, 1423776865, 2000544382, 2885797837, 3031315274, 730632025, 2472647826, 1863057327, 3018869246, 1979648458, 4026242931, 1418679383, 2270199421, 1219987712, 1288903000, 1076343333, 1366377560, 1446591244, 935998549, 1952968400, 2153547079, 1310677001, 3159517854, 1282178627, 954478089, 2551633621, 2985972637, 988112783, 2133689346, 1943961097, 2163443170, 2608905213, 1050699869, 2893098423, 1062168732, 488415722, 2500092199, 1502474515, 2934590505, 473148197, 2079619430, 3808136485, 2620793363, 964427302, 3289688385, 2318121737, 3449800372, 920396899, 2286521728, 1642766635

val_loss: 34.15315628051758


Step: 85 ___________________________________________
val_loss: 49.120059967041016


Step: 86 ___________________________________________
val_loss: 35.18827819824219


Step: 87 ___________________________________________
val_loss: 1.790232539176941


Step: 88 ___________________________________________
val_loss: 2.5693390369415283


Step: 89 ___________________________________________
val_loss: 1411.2923583984375


Step: 90 ___________________________________________
val_loss: 32.6001091003418


Step: 91 ___________________________________________
val_loss: 2652.098876953125


Step: 92 ___________________________________________
val_loss: 0.5671719908714294


Step: 93 ___________________________________________
val_loss: 2343.5458984375


Step: 94 ___________________________________________
val_loss: 0.038242340087890625


Step: 95 ___________________________________________
val_loss: 1.4079171419143677


Step: 96 ___________________________________________

In [16]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 454ms/step - loss: 25848.7344 - val_loss: 13359.6982
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 23590.8047 - val_loss: 21584.4277
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 23440.5430 - val_loss: 4345.1299
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 24261.5352 - val_loss: 12649.2979
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 20872.9824 - val_loss: 3563.9180
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 20702.8672 - val_loss: 1856.2262
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 18196.9297 - val_loss: 1947.6979
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 18005.3438 - val_loss: 2010.9310
Epoch 9/10000
1/1 [==============================] - 0s 15ms/step - loss: 17593.4316 - val_loss: 1799.7578
Epoch 10/10000
1/1 [=============

Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 6143.3657 - val_loss: 14.5355
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 6062.0103 - val_loss: 16.8469
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 5982.1357 - val_loss: 19.5437
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 5903.7148 - val_loss: 22.6080
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 5826.7217 - val_loss: 26.0225
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 5751.1294 - val_loss: 29.7844
Epoch 84/10000
1/1 [==============================] - 0s 14ms/step - loss: 5676.9116 - val_loss: 33.8772
Epoch 85/10000
1/1 [==============================] - 0s 14ms/step - loss: 5604.0957 - val_loss: 38.1723
Epoch 86/10000
1/1 [==============================] - 0s 14ms/step - loss: 5532.5186 - val_loss: 43.0483
Epoch 87/10000
1/1 [==============================] - 0

Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 2746.8120 - val_loss: 705.1409
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 2726.6389 - val_loss: 715.8264
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 2706.8130 - val_loss: 726.4995
Epoch 158/10000
1/1 [==============================] - 0s 14ms/step - loss: 2687.3499 - val_loss: 737.0981
Epoch 159/10000
1/1 [==============================] - 0s 14ms/step - loss: 2668.2246 - val_loss: 747.6858
Epoch 160/10000
1/1 [==============================] - 0s 15ms/step - loss: 2649.4324 - val_loss: 758.2369
Epoch 161/10000
1/1 [==============================] - 0s 14ms/step - loss: 2630.9727 - val_loss: 768.7473
Epoch 162/10000
1/1 [==============================] - 0s 14ms/step - loss: 2612.8354 - val_loss: 779.2219
Epoch 163/10000
1/1 [==============================] - 0s 14ms/step - loss: 2595.0171 - val_loss: 789.6569
Epoch 164/10000
1/1 [================

1/1 [==============================] - 0s 14ms/step - loss: 1891.0081 - val_loss: 1376.9668
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 1885.5850 - val_loss: 1383.6698
Epoch 233/10000
1/1 [==============================] - 0s 14ms/step - loss: 1880.2145 - val_loss: 1390.3271
Epoch 234/10000
1/1 [==============================] - 0s 14ms/step - loss: 1874.8643 - val_loss: 1398.2531
Epoch 235/10000
1/1 [==============================] - 0s 15ms/step - loss: 1869.4153 - val_loss: 1435.2990
Epoch 236/10000
1/1 [==============================] - 0s 14ms/step - loss: 1863.1740 - val_loss: 1387.1705
Epoch 237/10000
1/1 [==============================] - 0s 13ms/step - loss: 1855.8170 - val_loss: 1370.2382
Epoch 238/10000
1/1 [==============================] - 0s 14ms/step - loss: 1855.2302 - val_loss: 1435.9863
Epoch 239/10000
1/1 [==============================] - 0s 15ms/step - loss: 1850.0160 - val_loss: 1425.7594
Epoch 240/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 1650.6392 - val_loss: 1753.3981
Epoch 308/10000
1/1 [==============================] - 0s 15ms/step - loss: 1650.0065 - val_loss: 1746.8481
Epoch 309/10000
1/1 [==============================] - 0s 14ms/step - loss: 1647.0435 - val_loss: 1754.7177
Epoch 310/10000
1/1 [==============================] - 0s 14ms/step - loss: 1645.3032 - val_loss: 1756.3724
Epoch 311/10000
1/1 [==============================] - 0s 14ms/step - loss: 1643.6082 - val_loss: 1759.7174
Epoch 312/10000
1/1 [==============================] - 0s 14ms/step - loss: 1641.9393 - val_loss: 1762.5739
Epoch 313/10000
1/1 [==============================] - 0s 14ms/step - loss: 1640.2900 - val_loss: 1765.5375
Epoch 314/10000
1/1 [==============================] - 0s 14ms/step - loss: 1639.2502 - val_loss: 1769.5474
Epoch 315/10000
1/1 [==============================] - 0s 14ms/step - loss: 1637.0726 - val_loss: 1771.0309
Epoch 316/10000
1/1 [=======================

...

1/1 [==============================] - 0s 14ms/step - loss: 607.9321 - val_loss: 0.0102
Epoch 2403/10000
1/1 [==============================] - 0s 13ms/step - loss: 613.2028 - val_loss: 0.4674
Epoch 2404/10000
1/1 [==============================] - 0s 14ms/step - loss: 613.1425 - val_loss: 0.0237
Epoch 2405/10000
1/1 [==============================] - 0s 14ms/step - loss: 612.3525 - val_loss: 0.4454
Epoch 2406/10000
1/1 [==============================] - 0s 14ms/step - loss: 606.8611 - val_loss: 0.0292
Epoch 2407/10000
1/1 [==============================] - 0s 14ms/step - loss: 619.8057 - val_loss: 0.0454
Epoch 2408/10000
1/1 [==============================] - 0s 14ms/step - loss: 613.1205 - val_loss: 0.2137
Epoch 2409/10000
1/1 [==============================] - 0s 14ms/step - loss: 611.8821 - val_loss: 0.0245
Epoch 2410/10000
1/1 [==============================] - 0s 14ms/step - loss: 610.2056 - val_loss: 0.1801
Epoch 2411/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 14ms/step - loss: 630.4001 - val_loss: 0.3739
Epoch 2481/10000
1/1 [==============================] - 0s 13ms/step - loss: 610.7208 - val_loss: 0.0279
Epoch 2482/10000
1/1 [==============================] - 0s 14ms/step - loss: 610.3807 - val_loss: 0.0550
Epoch 2483/10000
1/1 [==============================] - 0s 15ms/step - loss: 615.1893 - val_loss: 0.4902
Epoch 2484/10000
1/1 [==============================] - 0s 14ms/step - loss: 609.3205 - val_loss: 0.2672
Epoch 2485/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.5856 - val_loss: 0.0745
Epoch 2486/10000
1/1 [==============================] - 0s 14ms/step - loss: 611.3227 - val_loss: 0.0899
Epoch 2487/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.0421 - val_loss: 0.1658
Epoch 2488/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.3296 - val_loss: 0.0472
Epoch 2489/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 14ms/step - loss: 614.8161 - val_loss: 0.0162
Epoch 2559/10000
1/1 [==============================] - 0s 14ms/step - loss: 624.3401 - val_loss: 0.0464
Epoch 2560/10000
1/1 [==============================] - 0s 15ms/step - loss: 612.1651 - val_loss: 0.1948
Epoch 2561/10000
1/1 [==============================] - 0s 14ms/step - loss: 612.6129 - val_loss: 0.0356
Epoch 2562/10000
1/1 [==============================] - 0s 14ms/step - loss: 612.8863 - val_loss: 0.1062
Epoch 2563/10000
1/1 [==============================] - 0s 13ms/step - loss: 618.9223 - val_loss: 0.4031
Epoch 2564/10000
1/1 [==============================] - 0s 14ms/step - loss: 606.6431 - val_loss: 0.1663
Epoch 2565/10000
1/1 [==============================] - 0s 14ms/step - loss: 616.5966 - val_loss: 0.2326
Epoch 2566/10000
1/1 [==============================] - 0s 14ms/step - loss: 618.1055 - val_loss: 0.4223
Epoch 2567/10000
1/1 [==============================] - 0s 14ms/step - l

1/1 [==============================] - 0s 14ms/step - loss: 617.6713 - val_loss: 0.2550
Epoch 2637/10000
1/1 [==============================] - 0s 15ms/step - loss: 626.6517 - val_loss: 1.3273
Epoch 2638/10000
1/1 [==============================] - 0s 14ms/step - loss: 625.1296 - val_loss: 1.0755
Epoch 2639/10000
1/1 [==============================] - 0s 14ms/step - loss: 609.8079 - val_loss: 0.2442
Epoch 2640/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.8929 - val_loss: 0.1283
Epoch 2641/10000
1/1 [==============================] - 0s 14ms/step - loss: 609.8715 - val_loss: 0.0892
Epoch 2642/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.8318 - val_loss: 0.0542
Epoch 2643/10000
1/1 [==============================] - 0s 14ms/step - loss: 608.7392 - val_loss: 0.0600
Epoch 2644/10000
1/1 [==============================] - 0s 14ms/step - loss: 610.5724 - val_loss: 0.1991
Epoch 2645/10000
1/1 [==============================] - 0s 15ms/step - l

1/1 [==============================] - 0s 14ms/step - loss: 617.5400 - val_loss: 0.1967
Epoch 2715/10000
1/1 [==============================] - 0s 22ms/step - loss: 611.6800 - val_loss: 0.1903
Epoch 2715: early stopping


In [17]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[157.621](test_target) - [[132.65071]](prediction) = 24.970288940429697


In [18]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [19]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
0,-1.822242,1.867029,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.775628,-0.201120,-1.815323,0.526683,-0.930559,-1.830748,-0.717766,-1.595141,-1.449699,-1.742940,-1.876401
1,-1.111250,1.045116,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.360828,-0.500302,-1.237890,0.084565,-0.878366,-1.206230,-0.234890,-1.072139,-1.202688,-0.886900,-1.083744
2,-0.496667,0.669687,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.329560,-0.936412,-0.316133,-0.608483,-0.791047,-0.216873,0.512386,-0.575192,-0.562528,-0.111729,-0.356662
3,0.022494,-0.314227,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,0.985189,-1.147099,0.605270,-1.179019,-0.609577,0.739858,1.151509,-0.111371,0.277568,1.051082,0.281205
4,0.447218,-0.509271,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,1.295721,-0.770013,1.181663,-1.253612,-0.274936,1.266505,1.309931,0.266106,0.578301,1.267334,0.536260
5,0.778494,-0.966548,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.950821,0.557198,1.068386,-0.458833,0.271896,0.965612,0.615100,0.578397,0.842315,1.081801,0.575958
6,1.017308,-0.701091,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.635300,1.743731,0.391276,1.424901,1.439997,0.034218,-0.949421,0.802429,-0.207105,-0.149082,0.400740
7,1.164646,-1.090695,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.789536,1.254016,0.122752,1.463797,1.772593,0.247658,-1.686849,1.706911,1.723837,-0.509565,1.522643


train_target:


,Amapá - Consumo de Cimento (t)
0,80.008
1,104.197
2,115.540
3,135.046
4,116.366
5,112.729
6,151.542
7,161.970


test_input:


...

,Amapá - IDH,Amapá - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado
17,-1.683117,0.92733,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.388764,0.425197,-1.178467,-0.682869,0.106644,0.829078,-1.095947,0.821846,-1.025345,-1.444524,0.649078


test_target:


,Amapá - Consumo de Cimento (t)
17,157.621


1/1 [==============================] - 0s 33ms/step
Error: 24.970288940429697




[158.8890838623047,
 191.5744171142578,
 210.6872100830078,
 212.86131286621094,
 208.41612243652344,
 201.33340454101562,
 183.9803466796875,
 185.24334716796875,
 145.91644287109375,
 132.6507110595703]

In [20]:
display(targets)
display(predictions)

[261.12,
 250.519,
 227.243,
 198.0696,
 166.85039999999998,
 135.021,
 123.347,
 120.153,
 133.523,
 157.621]

[158.8890838623047,
 191.5744171142578,
 210.6872100830078,
 212.86131286621094,
 208.41612243652344,
 201.33340454101562,
 183.9803466796875,
 185.24334716796875,
 145.91644287109375,
 132.6507110595703]

In [21]:
mae = mean_absolute_error(predictions, targets)
mae

46.34885544433594

In [22]:
porcentage = mae/np.mean(targets)
porcentage

0.2613460269874542